## 1. Import Libraries

In [21]:
# Feature_Engineering
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import *
from pyspark.sql.functions import col, count, sum
# Modeling
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import linear_model, datasets
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
import numpy as np
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import numpy as np
import matplotlib.pyplot as plt

## 2. Load Data and Convert to Spark Data Frame

In [22]:
# read files
sc.textFile("loan.csv").take(1)

[u'id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m']

In [23]:
# load data as dataframe
loan_df=spark.read.csv("loan.csv",header=True)

In [24]:
loan_df.rdd.getNumPartitions()
type(loan_df)

pyspark.sql.dataframe.DataFrame

## 3. Create response variable and features
### 3.1 Remove some columns based on EDA results

In [25]:
loan_df2=loan_df.select(
    loan_df.loan_amnt.cast("float"),
    loan_df.funded_amnt.cast("float"),
    loan_df.funded_amnt_inv.cast("float"),
    loan_df.int_rate.cast("float"),  #convert to float
    loan_df.installment.cast("float"), #convert to float
    'grade',
    'sub_grade',   # NEED TO BE DUMMIED
    'emp_length',
    'home_ownership', #group & make it dummy, done 
    loan_df.annual_inc.cast("float"),
    # 'issue_d',  #why comment out?
    'loan_status', 
    # response variable (pending for change after group discussion)
    'purpose',  #make it dummy
#     'zip_code', #interesting, worth dummying, might run into problems tho
#     'addr_state', #dummy
    'verification_status',
#     'initial_list_status',
    loan_df.dti.cast("float"),#float done
    loan_df.delinq_2yrs.cast("integer"),  
    #take out NAs, 2? 1, 3, mostly 0
    # 'earliest_cr_line',
    loan_df.inq_last_6mths.cast("integer"), #similar to delingq_2yr
    loan_df.open_acc.cast("integer"), 
    loan_df.pub_rec.cast("integer"), #what is it? 0,1
    loan_df.revol_bal.cast("float"), #done
    loan_df.revol_util.cast("float"),#done
    loan_df.total_acc.cast("integer"),
    #loan_df.last_credit_pull_d.cast("integer"), #date
    loan_df.acc_now_delinq.cast("integer"),#ok
    loan_df.tot_coll_amt.cast("float"), #done
    loan_df.tot_cur_bal.cast("float"), #done
    loan_df.total_rev_hi_lim.cast("integer") #what is this
)

### 3.2 Create response variable and remove rows with no valid response variable 

In [26]:
# needs to fix that, encoding doesn't seem to be right
def whetherpaid(x):
    if x in ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']:
        return 1
    elif x in ['Does not meet the credit policy. Status:Fully Paid', 'Fully Paid']:
        return 0
    else:
        return -1

In [27]:
paidflag = udf(lambda x: whetherpaid(x))

In [28]:
#subject to change
loan_df3 = loan_df2.withColumn(
    'paid_flag',
    paidflag('loan_status').cast("integer")
        ).where("paid_flag != -1").drop('loan_status')

In [29]:
loan_df3.printSchema()

root
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: float (nullable = true)
 |-- funded_amnt_inv: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- purpose: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integ

### 3.3 Add more features
#### 3.3.1 Create a numeric feature for "emp_length"

In [30]:
loan_df3.select('emp_length').distinct().show()

+----------+
|emp_length|
+----------+
|   9 years|
|   5 years|
|    1 year|
|       n/a|
|   2 years|
|   7 years|
|   8 years|
|   4 years|
|   6 years|
|   3 years|
| 10+ years|
|  < 1 year|
+----------+



In [31]:
import re
def convert_to_int(s):
    s = re.sub('\\D', '', s)  #remove any non-digital character
    #\d matches any digital, #\D matches any non-digital
    try:
        return s
    except ValueError:
        return None

emp_to_num = udf(convert_to_int)
loan_df4 = loan_df3.withColumn('emp_len',emp_to_num('emp_length').cast('integer')).drop('emp_length')

In [32]:
loan_df4.select('emp_len').distinct().show()

+-------+
|emp_len|
+-------+
|   null|
|      1|
|      6|
|      3|
|      5|
|      9|
|      4|
|      8|
|      7|
|     10|
|      2|
+-------+



#### 3.3.2 Create numeric variable for grade 

In [33]:
# count how many nulls in each column
def count_null(c):
    return sum(col(c).isNull().cast("integer")).alias(c)

exprs = [count_null(c) for c in loan_df4.columns[9:19]]
loan_df4.agg(*exprs).show()

+-------+-------------------+---+-----------+--------------+--------+-------+---------+----------+---------+
|purpose|verification_status|dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|
+-------+-------------------+---+-----------+--------------+--------+-------+---------+----------+---------+
|      0|                  0|252|        225|           198|     195|    161|      102|       314|       88|
+-------+-------------------+---+-----------+--------------+--------+-------+---------+----------+---------+



#### 3.3.3 Create numeric variable for grade 

In [37]:
loan_df4.select('home_ownership').distinct().show()

+--------------+
|home_ownership|
+--------------+
|           OWN|
|          RENT|
|      MORTGAGE|
|           ANY|
|         OTHER|
|          NONE|
+--------------+



In [38]:
def home_ownership_func(x):
    if x in ['ANY','OTHER','NONE']:
        return 'Other'
    else: 
        return x

home_ownership = udf(home_ownership_func)  #fixed a function
loan_df5 = loan_df4.withColumn('home_ownership',home_ownership('home_ownership'))

In [13]:
loan_df5.select('home_ownership').groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN| 22282|
|         Other|   228|
|          RENT|107831|
|      MORTGAGE|126598|
+--------------+------+



In [40]:
loan_df5.select('home_ownership').distinct().show()

+--------------+
|home_ownership|
+--------------+
|           OWN|
|         Other|
|          RENT|
|      MORTGAGE|
+--------------+



#### 3.3.4 Add loan_inc_ratio feature 

In [41]:
def calculate_ratio(a, b):
    try:
        return a/float(b)
    except TypeError:
        return None
    except ZeroDivisionError:
        return None

ratio = udf(calculate_ratio) #define a udf_function ratio

In [42]:
loan_df6 = loan_df5.withColumn(
    'loan_inc_ratio',ratio('loan_amnt','annual_inc'
                          ).cast('float'))

In [43]:
loan_df7 = loan_df6.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

#### 3.3.5 Add feature instal_inc_ratio


In [44]:
def calculate_monthly_ratio(a, b):
    try:
        return a/(float(b)/12)
    except TypeError:
        return None
    except ZeroDivisionError:
        return None
    
monthly_ratio = udf(calculate_monthly_ratio)

In [45]:
loan_df8 = loan_df7.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

In [46]:
loan_df8.printSchema()

root
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: float (nullable = true)
 |-- funded_amnt_inv: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- purpose: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- paid_flag: integ

#### 3.3.6 Create dummy variables
* Use StringIndexer to encode a string column of labels to a column of label indices, and most frequent label gets index 0.

* Use OneHotEncoder to convert indices into dummy vectors

In [47]:
convert_list = [      
               'purpose',
               'grade',
               'sub_grade',
               'verification_status',
               'home_ownership',
                ]

for item in convert_list:
    indexer = StringIndexer(inputCol=item, outputCol=item + 'Index')    
    loan_df8 = indexer.fit(loan_df8).transform(loan_df8).drop(item)
    onehotenc = OneHotEncoder(inputCol=item + 'Index', outputCol=item+"-onehot", dropLast=False)
    # we can experiment with True
    loan_df8 = onehotenc.transform(loan_df8).drop(item+'Index')

#### 3.3.7 deal with missing values

In [50]:
# Filling NA values with 0
loan_df8 = loan_df8.fillna(0.0, ['tot_coll_amt','tot_cur_bal', 'total_rev_hi_lim'])
# Drop rows with NA
loan_df9 = loan_df8.dropna()

In [51]:
loan_df9.columns

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'paid_flag',
 'emp_len',
 'loan_inc_ratio',
 'instal_inc_ratio',
 'purpose-onehot',
 'grade-onehot',
 'sub_grade-onehot',
 'verification_status-onehot',
 'home_ownership-onehot']

In [52]:
loan_df9.select('home_ownership-onehot').show()

+---------------------+
|home_ownership-onehot|
+---------------------+
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[2],[1.0])|
|        (4,[1],[1.0])|
|        (4,[2],[1.0])|
|        (4,[2],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[0],[1.0])|
|        (4,[0],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
|        (4,[1],[1.0])|
+---------------------+
only showing top 20 rows



In [53]:
loan_df_final = loan_df9.select(
 'paid_flag',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'emp_len',
 'loan_inc_ratio',
 'instal_inc_ratio',
 'purpose-onehot',
 'grade-onehot',
 'sub_grade-onehot',
 'verification_status-onehot',
 'home_ownership-onehot'
)

In [24]:
loan_df_final.take(1)
loan_df_final.printSchema()

root
 |-- paid_flag: integer (nullable = true)
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: float (nullable = true)
 |-- funded_amnt_inv: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = false)
 |-- tot_cur_bal: float (nullable = false)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- emp_len: integer (nullable = true)
 |-- loan_inc_ratio: float (nullable = true)
 |-- instal_inc_ratio: float (nullable = true)
 |-- purpose-onehot: vector (nullable = true)
 |-- grade-on

## 4. Modeling

### 4.1 Instantiating Data

In [25]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [26]:
 # Split the data into training and test sets (30% held out for testing)
loan_df=loan_df_final.withColumnRenamed('paid_flag','label')
loan_df.head(2)

[Row(label=0, loan_amnt=5000.0, funded_amnt=5000.0, funded_amnt_inv=4975.0, int_rate=10.649999618530273, installment=162.8699951171875, annual_inc=24000.0, dti=27.649999618530273, delinq_2yrs=0, inq_last_6mths=1, open_acc=3, pub_rec=0, revol_bal=13648.0, revol_util=83.69999694824219, total_acc=9, acc_now_delinq=0, tot_coll_amt=0.0, tot_cur_bal=0.0, total_rev_hi_lim=0, emp_len=10, loan_inc_ratio=0.2083333283662796, instal_inc_ratio=0.006786249577999115, purpose-onehot=SparseVector(265, {1: 1.0}), grade-onehot=SparseVector(7, {0: 1.0}), sub_grade-onehot=SparseVector(35, {3: 1.0}), verification_status-onehot=SparseVector(3, {0: 1.0}), home_ownership-onehot=SparseVector(4, {1: 1.0})),
 Row(label=1, loan_amnt=2500.0, funded_amnt=2500.0, funded_amnt_inv=2500.0, int_rate=15.270000457763672, installment=59.83000183105469, annual_inc=30000.0, dti=1.0, delinq_2yrs=0, inq_last_6mths=5, open_acc=3, pub_rec=0, revol_bal=1687.0, revol_util=9.399999618530273, total_acc=4, acc_now_delinq=0, tot_coll_a

In [27]:
va=VectorAssembler(outputCol='features',inputCols=loan_df.columns[1:])
data=va.transform(loan_df).select('features','label')

In [28]:
data.head(3)

[Row(features=SparseVector(335, {0: 5000.0, 1: 5000.0, 2: 4975.0, 3: 10.65, 4: 162.87, 5: 24000.0, 6: 27.65, 8: 1.0, 9: 3.0, 11: 13648.0, 12: 83.7, 13: 9.0, 18: 10.0, 19: 0.2083, 20: 0.0068, 22: 1.0, 286: 1.0, 296: 1.0, 328: 1.0, 332: 1.0}), label=0),
 Row(features=SparseVector(335, {0: 2500.0, 1: 2500.0, 2: 2500.0, 3: 15.27, 4: 59.83, 5: 30000.0, 6: 1.0, 8: 5.0, 9: 3.0, 11: 1687.0, 12: 9.4, 13: 4.0, 18: 1.0, 19: 0.0833, 20: 0.002, 27: 1.0, 287: 1.0, 300: 1.0, 330: 1.0, 332: 1.0}), label=1),
 Row(features=SparseVector(335, {0: 2400.0, 1: 2400.0, 2: 2400.0, 3: 15.96, 4: 84.33, 5: 12252.0, 6: 8.72, 8: 2.0, 9: 2.0, 11: 2956.0, 12: 98.5, 13: 10.0, 18: 10.0, 19: 0.1959, 20: 0.0069, 26: 1.0, 287: 1.0, 303: 1.0, 329: 1.0, 332: 1.0}), label=0)]

In [29]:
data.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(335,[0,1,2,3,4,5...|    0|
|(335,[0,1,2,3,4,5...|    1|
|(335,[0,1,2,3,4,5...|    0|
+--------------------+-----+
only showing top 3 rows



In [30]:
# Creating Training and Test Data
data_sets=data.randomSplit([0.8,0.2])
data_train=data_sets[0].cache()
data_test=data_sets[1].cache()

### 4.2 Random Forest

#### 4.2.1 Initial model

In [31]:
# Fit random forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features")
rfmodel=rf.fit(data_train)

In [32]:
# Make predictions.
predictions = rfmodel.transform(data_test)
predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(335,[0,1,2,3,4,5...|    0|[15.9549480633131...|[0.79774740316565...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[14.1290173051074...|[0.70645086525537...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[15.6197347295825...|[0.78098673647912...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[17.5381664912275...|[0.87690832456137...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[17.1375076870517...|[0.85687538435258...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[16.0960367814478...|[0.80480183907239...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[16.2802870344192...|[0.81401435172096...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[17.0815210619768...|[0.85407605309884...|       0.0|
|(335,[0,1,2,3,4,5...|    0|[16.7394747907107...|[0.83697373953553...|       0.0|
|(335,[0,1,2,3,4

#### 4.2.2 Measure the performace: The Validation set approach

In [33]:
# Evaluate the model. metric: Area Under ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
bceval = BinaryClassificationEvaluator()
print (bceval.getMetricName() + ":" + str(bceval.evaluate(predictions)))

areaUnderROC:0.691452102764


In [34]:
# Evaluate the model. metric: Area Under PR
bceval.setMetricName("areaUnderPR")
print (bceval.getMetricName() + ":" + str(bceval.evaluate(predictions)))

areaUnderPR:0.322841141042


#### 4.2.3 OverSampling the data to solve unbalanced data problem

In [35]:
print 'training dataset fraud propotition:', float(data_train.filter('label=1').count())/float((data_train.filter('label=1').count() + data_train.filter('label=0').count()))

training dataset fraud propotition: 0.180889657867


In [36]:
print 'test dataset fraud propotition:', float(data_test.filter('label=1').count())/float((data_test.filter('label=1').count() + data_test.filter('label=0').count()))

test dataset fraud propotition: 0.179765039498


In [37]:
data_train_paid = data_train.filter(data_train.label == 0)
data_train_not_paid = data_train.filter(data_train.label == 1)

In [38]:
num_samples_train = data_train.count()
half_num_samples_train = num_samples_train / 2

In [39]:
data_train_paid = data_train_paid.sample(True, float(half_num_samples_train) / float(data_train_paid.count())) 
data_train_not_paid = data_train_not_paid.sample(True, float(half_num_samples_train) / float(data_train_not_paid.count()))

In [40]:
data_train_paid.count()

98328

In [41]:
data_train_not_paid.count()

98789

In [ ]:
data_train = data_train_paid.union(data_train_not_paid)  

##### 4.2.4 Parameter Tunning

In [ ]:
# n-fold validation and the results.
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
RandomForestClassifier(labelCol="label", featuresCol="features")

cv = CrossValidator().setEstimator(rf).setEvaluator(bceval).setNumFolds(5)
paramGrid = ParamGridBuilder().addGrid(rf.numTrees,[10,25]).addGrid(rf.maxDepth,[15,20]).build()
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(data_train)

#### 4.2.5 Measure the bset model performance

In [ ]:
# Measure the performace
print 'areaunderROC', BinaryClassificationEvaluator().evaluate(cvmodel.bestModel.transform(data_test))
print 'areunderPR', BinaryClassificationEvaluator().setMetricName("areaUnderPR").evaluate(cvmodel.bestModel.transform(data_test))

In [ ]:
# get predict,label tuple method 1
predict_label_list = predictions.select('label','prediction').collect()
predictionAndLabels = [(float(i[1]),float(i[0])) for i in predict_label_list]
metrics = MulticlassMetrics(sc.parallelize(predictionAndLabels))
print 'Confusion Matrix:/n', metrics.confusionMatrix().toArray()
print 'percision rate:', metrics.precision(1)
print 'recall rate:', metrics.recall(1)

#### 4.2.6 Feature Importance

In [ ]:
rfmodel=cvmodel.bestModel
importance_list = list(enumerate(rfmodel.featureImportances))
importance_list.sort(key=lambda tup: -tup[1])
top_importances = importance_list[:15]
top_importances

In [ ]:
# # visiualize the importance 
# std = np.std([tree.feature_importances_ for tree in forest.estimators_],
#              axis=0)
# indices = np.argsort(importances)[::-1]

# # Print the feature ranking
# print("Feature ranking:")

# for f in range(X.shape[1]):
#     print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# # Plot the feature importances of the forest
# plt.figure()
# plt.title("Feature importances")
# plt.bar(range(X.shape[1]), importances[indices],
#        color="r", yerr=std[indices], align="center")
# plt.xticks(range(X.shape[1]), indices)
# plt.xlim([-1, X.shape[1]])
# plt.show()

### 4.3 Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

lr=LogisticRegression()
evaluator=BinaryClassificationEvaluator()

cv=CrossValidator().setEstimator(lr).setEvaluator(evaluator).setNumFolds(5)
paramGrid=ParamGridBuilder().addGrid(lr.maxIter,[100]).addGrid(lr.regParam,[0.1,0.01,0.001,1]).build()

cv.setEstimatorParamMaps(paramGrid)
cvmodel=cv.fit(data_train)

In [ ]:
predictions=cvmodel.bestModel.transform(data_test)

In [ ]:
print predictions.show(1)

In [ ]:
score=evaluator.evaluate(cvmodel.bestModel.transform(data_test))
print evaluator.getMetricName()+":"+str(score)

In [ ]:
prediction_list = predictions.collect()
predictionAndLabels = []
for item in prediction_list:
    predictionAndLabels.append((float(item[4]), float(item[1])))

In [ ]:
metrics = MulticlassMetrics(sc.parallelize(predictionAndLabels))
print metrics.confusionMatrix().toArray()
print metrics.precision(1)
print metrics.recall(1)
print metrics.accuracy